In [ ]:
!pip install rdkit-pypi
!pip install openpyxl
!pip install rdkit-pypi avalon_framework

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for avalon_framework: filename=avalon_framework-1.8.2-py3-none-any.whl size=3863 sha256=6e116aff4716163682e4501cd4c1e556086a43e686f003baa3703da6dd6bc8bc
  Stored in directory: /root/.cache/pip/wheels/78/3f/5c/a65bfa8ce94f62739865cf30e5687272ee719961b4311d05e3
Successfully built avalon_framework


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import learning_curve

In [ ]:
# Upload the Excel file containing SMILES, CLASS and F/F data
uploaded = files.upload()

# Read the uploaded Excel file
df = pd.read_excel(next(iter(uploaded)))

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# Generate fingerprints using RDKit's Morgan fingerprint
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in mols]

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
print(class_counts)
print(f"Number of features in Morgan_fingerprints: {len(fingerprints[0])}")



Saving INPUT-CORRECT-ML-CLASS.xlsx to INPUT-CORRECT-ML-CLASS (3).xlsx
Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
fingerprints_list = [list(fp.ToBitString()) for fp in fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(2048)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-Morgan-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_Morgan_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in new_mols]

print(f"Number of features in new_Morgan_fingerprints: {len(new_Morgan_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_Morgan_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-Morgan-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)

# Print or use the predictions as needed
#print("Predictions for each model:")
#print(predictions_df)


    Class                                       Fingerprints
0       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
..    ...                                                ...
58      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
59      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
60      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_Morgan_fingerprints: 2048


# MACCS


In [ ]:
from rdkit.Chem import MACCSkeys

# Read the uploaded Excel file
df = pd.read_excel("INPUT-CORRECT-ML-CLASS.xlsx")

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
maccs_fingerprints = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
print(class_counts)
print(f"Number of features in Maccs_fingerprints: {len(maccs_fingerprints[0])}")

Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=CC(O)=C1O', 'OC1=C(F)C=CC=C1O', 'OC1=CC(O)=CC(O)=C1', 'OC1=C(N)C=CC=C1

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
maccs_fingerprints_list = [list(fp.ToBitString()) for fp in maccs_fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": maccs_fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(167)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(maccs_fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-Maccs-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_maccs_fingerprints = [MACCSkeys.GenMACCSKeys(mol) for mol in new_mols]

print(f"Number of features in new_maccs_fingerprints: {len(new_maccs_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_maccs_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-Maccs-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)

# Print or use the predictions as needed
#print("Predictions for each model:")
#print(predictions_df)

    Class                                       Fingerprints
0       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
..    ...                                                ...
58      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
59      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
60      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_maccs_fingerprints: 167


# Daylight

In [ ]:
from rdkit import Chem
from rdkit.Chem import DataStructs

# Read the uploaded Excel file
df = pd.read_excel("INPUT-CORRECT-ML-CLASS.xlsx")

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
Daylight_fingerprints = [Chem.RDKFingerprint(mol) for mol in mols]

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
#print(class_counts)
print(f"Number of features in Daylight_fingerprints: {len(Daylight_fingerprints[0])}")

Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=CC(O)=C1O', 'OC1=C(F)C=CC=C1O', 'OC1=CC(O)=CC(O)=C1', 'OC1=C(N)C=CC=C1

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
Daylight_fingerprints_list = [list(fp.ToBitString()) for fp in Daylight_fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": Daylight_fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(2048)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(Daylight_fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-Daylight-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_Daylight_fingerprints = [Chem.RDKFingerprint(mol) for mol in new_mols]

print(f"Number of features in new_Daylight_fingerprints: {len(new_Daylight_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_Daylight_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-Daylight-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)


    Class                                       Fingerprints
0       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       0  [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
3       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       0  [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, ...
..    ...                                                ...
58      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
59      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
60      0  [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_Daylight_fingerprints: 2048


# Atompairs

In [ ]:
from rdkit.Chem import rdMolDescriptors

# Read the uploaded Excel file
df = pd.read_excel("INPUT-CORRECT-ML-CLASS.xlsx")

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
atom_pairs_fingerprints = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol) for mol in mols]

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
print(class_counts)
print(f"Number of features in Atompairs_fingerprints: {len(atom_pairs_fingerprints[0])}")

Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=CC(O)=C1O', 'OC1=C(F)C=CC=C1O', 'OC1=CC(O)=CC(O)=C1', 'OC1=C(N)C=CC=C1

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
Atompairs_fingerprints_list = [list(fp.ToBitString()) for fp in atom_pairs_fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": Atompairs_fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(2048)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(Daylight_fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-atompairs-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_atom_pairs_fingerprints = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol) for mol in new_mols]

print(f"Number of features in new_atom_pairs_fingerprints: {len(new_atom_pairs_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_Daylight_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-atompairs-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)


    Class                                       Fingerprints
0       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
..    ...                                                ...
58      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
59      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
60      0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_atom_pairs_fingerprints: 2048


# Avalon

In [ ]:
from rdkit import Chem
from rdkit.Avalon import pyAvalonTools

# Read the uploaded Excel file
df = pd.read_excel("INPUT-CORRECT-ML-CLASS.xlsx")

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
Avalon_fingerprints = [pyAvalonTools.GetAvalonFP(mol) for mol in mols]

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
print(class_counts)
print(f"Number of features in Avalon_fingerprints: {len(Avalon_fingerprints[0])}")

Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=CC(O)=C1O', 'OC1=C(F)C=CC=C1O', 'OC1=CC(O)=CC(O)=C1', 'OC1=C(N)C=CC=C1

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
Avalon_fingerprints_list = [list(fp.ToBitString()) for fp in Avalon_fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": Avalon_fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(512)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(Avalon_fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-avalon-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_Avalon_fingerprints = [pyAvalonTools.GetAvalonFP(mol) for mol in new_mols]

print(f"Number of features in new_avalon_fingerprints: {len(new_Avalon_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_Avalon_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-avalon-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)


    Class                                       Fingerprints
0       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...
2       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
3       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
4       0  [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ...
..    ...                                                ...
58      0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...
59      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
60      0  [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_avalon_fingerprints: 512


# Torsion

In [ ]:
from rdkit.Chem import rdMolDescriptors

# Read the uploaded Excel file
df = pd.read_excel("INPUT-CORRECT-ML-CLASS.xlsx")

# Define column names for better clarity
column_names = ["Cmpd Lab", "Smiles", "dFF", "Class"]

# Rename the DataFrame columns
df.columns = column_names

# Extract columns using descriptive variable names
smiles_column = df['Smiles']
ff_column = df['dFF']

# Convert SMILES column to a list
smiles_list = smiles_column.to_list()

# Convert F/F columns to numeric arrays
ff_values = ff_column.to_numpy().astype(float)

# Create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
torsion_fingerprints = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol) for mol in mols]
len(mols)

# Create 'labels' array based on positive and negative numbers
df["Class"] = (df["dFF"] > 0.4).astype(int)

# Print the length of the molecule and fingerprint lists
print(f"Number of molecules: {len(mols)}")
print(smiles_list)
print(f"Number of fingerprints: {len(fingerprints)}")
print(df["Class"])
#print(labels) #where are labels defined?
class_counts = df["Class"].value_counts()
print(class_counts)
print(f"Number of features in torsion_fingerprints: {len(torsion_fingerprints[0])}")

Number of molecules: 63
['OC1=CC=CC(O)=C1', 'SC1=CC=CC=C1[S-]', 'OC1=CC=NC=C1O', 'OC1=C2C(C=CC=C2O)=CC=C1', '[NH3+]CCC1=CNC2=CC=C(C=C12)O', 'O=C1C([O-])=CC=CC=C1', 'OC1=CC=C(C(C)(C)C)C=C1O', 'O=C(C)NCCC1=CNC2=CC=C(C=C12)OC', '[NH3+]C[C@H](O)C1=CC=C(O)C(O)=C1', 'OC1=CC2=C(C=C1O)NC=C2', 'OC1=CC(Cl)=CC=C1O', '[O-]C([C@H](O)[C@@H](O)C([O-])=O)=O', 'OC1=C(O)C=CC(C([O-])=O)=C1', 'OCC(O)C(O)C(O)C(O)CO', 'OC1=C(O)C=CC(O)=C1', 'O=CC1=C(C=O)C=CC=C1', 'CCC1=CC=C(O)C(O)=C1', ' [NH3+]CCC1=CC=C(O)C(O)=C1', 'COC1=CC=CC=C1OC', 'O=C(C(C(C)(C)C)=CC(C(C)(C)C)=C1)C1=O', 'OC1=C(O)C=CC=C1OC', 'OC1=CC(N)=CC=C1O', 'OC1=C(C=O)C=CC=C1', 'OC1=C(CO)C=CC=C1', 'OC1=CC=CC=C1OC', 'OCC1=C(CO)C=CC=C1', 'OC1=CC([N+]([O-])=O)=CC=C1[O-]', 'CC(C1=CC=C(O)C(O)=C1)=O', 'OC1=C(O)C=CC(C(OC)=O)=C1', 'O=C(CO)C1=CC=CC=C1', 'OC(C([O-])=O)C1=CC=CC=C1', 'OC1=CC(C#N)=CC=C1O', 'OC1=C(O)C=CC(OC)=C1', 'OC1=CC=C(C=O)C=C1O', 'OC1=C(Cl)C=CC=C1O', 'OC1=CC=CN=C1O', 'OC1=CC=CC(O)=C1O', 'OC1=C(F)C=CC=C1O', 'OC1=CC(O)=CC(O)=C1', 'OC1=C(N)C=CC=C1

In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 45
np.random.seed(seed_value)

# Convert RDKit fingerprints to a list of lists
torsion_fingerprints_list = [list(fp.ToBitString()) for fp in torsion_fingerprints]

# Create a new DataFrame with molecular fingerprints and class labels
new_df = pd.DataFrame(data={"Class": df["Class"], "Fingerprints": torsion_fingerprints_list})

# Split the data into features (X) and labels (y)
X = pd.DataFrame(new_df["Fingerprints"].to_list(), columns=[f"Feature_{i}" for i in range(2048)]) #change range of i according to fingerprint
y = new_df["Class"]

print(new_df)

# Use fingerprints directly for features
features = np.array(torsion_fingerprints_list)

# Initialize lists to store evaluation metrics and confusion matrices
confusion_matrix_list = []
metrics_list = []

num_models = 200

# Initialize variables to track the 5 best models within the specified f1 score range
top_models_indices = []
top_models_f1_scores = []
top_models_accuracies = []
top_models = []  # Store the models

f1_range_lower = 0.75
f1_range_upper = 0.9

# Run the model 200 times with different random states
for i in range(num_models):
    # Shuffle the data using a random number generator
    permutation = np.random.permutation(len(features))
    shuffled_features = features[permutation]
    shuffled_labels = new_df["Class"][permutation]

    # Split the shuffled features and labels into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(shuffled_features, shuffled_labels, test_size=0.20, random_state=i)

    # Initialize and train the SVC model
    model = SVC(probability=True)
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append metrics to the list
    metrics_list.append({"Model": i + 1, "Accuracy": accuracy, "F1 Score": f1, "Recall": recall})
    confusion_matrix_list.append(cm)

    # Update top models list if f1 score is within the specified range
    if f1_range_lower <= f1 <= f1_range_upper:
        top_models_indices.append(i)
        top_models_f1_scores.append(f1)
        top_models_accuracies.append(accuracy)
        top_models.append(model)  # Store the model

# Sort top models based on f1 score in descending order
sorted_top_models_indices = [idx for idx, _ in sorted(enumerate(top_models_f1_scores), key=lambda x: x[1], reverse=True)]
top_models_indices = [top_models_indices[idx] for idx in sorted_top_models_indices[:5]]

# Save the indices, f1 scores, and accuracies to a file for the top 5 models
top_models_info = pd.DataFrame({"Model Index": top_models_indices[:5], "F1 Score": top_models_f1_scores[:5], "Accuracy": top_models_accuracies[:5]})
top_models_info.to_csv("class-topmodels_info-seed45-f1range-torsion-t4.csv", index=False)

#new molecule features
df = pd.read_csv('INPUT-NEW-MOLS-correct.csv')
new_smiles_list = df["Smiles"]
new_molecule_names = df["Cmpd Label"]
new_mols = [Chem.MolFromSmiles(smiles) for smiles in new_smiles_list]
new_torsion_fingerprints = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol) for mol in new_mols]

print(f"Number of features in new_torsion_fingerprints: {len(new_torsion_fingerprints[0])}")

# Initialize predictions_df DataFrame
predictions_df = pd.DataFrame({"Molecule Names": new_molecule_names, "Molecule SMILES": new_smiles_list})

# Iterate over the top 5 models
for idx, model in enumerate(top_models[:5]):
    # Use the model to predict classes for new molecules
    predictions = model.predict(new_torsion_fingerprints)

    # Save predictions to the DataFrame
    predictions_df[f'Model_{idx + 1}_Predictions'] = predictions

    # Save predictions to a CSV file
    model_predictions_filename = f"class-predictions-seed45-torsion-t4-Model{top_models_indices[idx] + 1}.csv"
    predictions_df[['Molecule Names', f'Model_{idx + 1}_Predictions']].to_csv(model_predictions_filename, index=False)


    Class                                       Fingerprints
0       0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
2       0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       1  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
..    ...                                                ...
58      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
59      1  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
60      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
61      0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
62      0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[63 rows x 2 columns]
Number of features in new_torsion_fingerprints: 2048
